# PROJECT 1: EXPLORATORY DATA ANALYSIS

## Project Name: 

SOUNDCLOUD ANALYSIS

## Assignment Table

| ID            | Fullname              | Contribution rate |
| :-----------: | :-------------------  | :-----------:     |
| 20127323      | Võ Nhật Tân           |                   |
| 20127447      | Ngô Đức Bảo           |                   |
| 20127275      | Lê Nguyễn Nhật Phú    |                   |
| 20127681      | Nguyễn Thiên Phúc     |                   |



# Import Libraries

In [15]:
import numpy as np
import time
import pandas as pd
import random
import requests
import requests_cache
from bs4 import BeautifulSoup
import os.path
from concurrent.futures import ThreadPoolExecutor, process, wait

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
requests_cache.install_cache('cache', expire_after=None)

In [111]:
SLEEP_TIME = 2
SCROLL_PAUSE_TIME = 0.5

## Data Collection

In [79]:
def asynchronousGetWebData(listData, getDataFunc, *argv, max_screens = 3,):
    '''
    Input:
        - listData: list of urls which need to parse the content to the useful information 
        - getDataFunc - func: Fucntion supports parse each item in listData to get information
            * First parameter must be the item in list data
            * The next parameter is saved in *argv
        - max_screens - int: Maximum screens that open at the same time
        
    Output:
        - Returns list of result data
    '''
    result_value = []
    
    for item_index in range(0, len(listData), max_screens):
        start = item_index
        end = item_index + max_screens
        if end > len(listData):
            end = len(listData)
            
        tmpData = listData[start:end]
        threadList = []
        with ThreadPoolExecutor() as executor:
            for url in tmpData:
                threadList.append(executor.submit(getDataFunc, url, *argv))
                
        wait(threadList)
        
        for thread in threadList:
            if thread.done() and not thread.cancelled():
                result_value.append(thread.result())
    return result_value
    

### Collect Users

In [63]:
urlList = [ 'https://soundcloud.com/user-593335594/sets/is-500-songs-the-playlist', \
            'https://soundcloud.com/user-593335594/sets/amazing-artistry-2',\
            'https://soundcloud.com/user-593335594/sets/playlist-that-wont-crash-my',\
            'https://soundcloud.com/user-593335594/sets/my-personal-favorites',\
            'https://soundcloud.com/user-593335594/sets/amazing',\
            'https://soundcloud.com/user-593335594/sets/i-found-more-music',\
            'https://soundcloud.com/user-593335594/sets/chivalry-is-dead',\
            'https://soundcloud.com/idla/sets/old-songs',\
            'https://soundcloud.com/idla/sets/pop-playlist-2020',\
            'https://soundcloud.com/idla/sets/lost-in-the-blues',\
            'https://soundcloud.com/user987150052/sets/hot-playlist',\
            'https://soundcloud.com/user61185041/sets/hot-playlist',\
            'https://soundcloud.com/discover/sets/charts-top:all-music:vn',\
            'https://soundcloud.com/discover/sets/charts-trending:danceedm:vn',\
            'https://soundcloud.com/bytufekci/sets/top-100-songs-of-2020']

In [64]:
def getAllUserURL(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    userProfileUrl = set()
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()
    except:
        pass

    #Scroll to the bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'trackItem__username')))
    newUserUrl = driver.find_elements(By.CLASS_NAME, 'trackItem__username')
    for user in newUserUrl:
        userProfileUrl.add(user.get_attribute('href'))
    driver.close()
    return userProfileUrl

In [67]:
userUrlList = []
if os.path.isfile('linkusercaches.txt'):
    with open('linkusercaches.txt', 'r') as f:
        userUrlList = f.read().splitlines()

else:
    data = asynchronousGetWebData(urlList, getAllUserURL)
    userUrlList = []
    for item in data:
        userUrlList = userUrlList + list(item)
    userUrlList = set(userUrlList)
    with open(r'linkusercaches.txt', 'w') as fp:
        for item in userUrlList:
            # write each item on a new line
            fp.write("%s\n" % item)
            
print(f'The number of profile link: {len(userUrlList)}')

The number of profile link: 1354


In [75]:
def get_user_info(url):    
#     #create a list of users using dataframes
#     users = pd.DataFrame(columns=['userID', 'username', 'Vertified', 'userUrl', 'followers', 'following', 'tracks', 'playlistIDs'])

    driver = webdriver.Chrome()
    driver.maximize_window()

    driver.get(url)

   
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

    username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'h2.profileHeaderInfo__userName'))).text

    isVertified = driver.find_element(By.CSS_SELECTOR, 'h2.profileHeaderInfo__userName>div').text
    if len(isVertified) > 0: username = username.strip()[:-len(isVertified.strip())].strip()
    userID = url.split('/')[-1]
    userurl = url
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'infoStats__value.sc-font-light')))
    info = driver.find_elements(By.CLASS_NAME, 'infoStats__value.sc-font-light')
    followers = info[0].text
    following = info[1].text
    tracks = info[2].text
    #go to user's sets which have the url https://soundcloud.com/{userID}/sets
    driver.get(f'https://soundcloud.com/{userID}/sets')
    playlistIDs = []

    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".emptyNetworkPage__headline, .sc-link-primary.soundTitle__title.sc-link-dark.sc-text-h4")))

    if not driver.find_elements(By.CLASS_NAME, 'emptyNetworkPage__headline'):
        playlist = driver.find_elements(By.CLASS_NAME, 'sc-link-primary.soundTitle__title.sc-link-dark.sc-text-h4')
        if len(playlist) != 0:
            for p in playlist:
                playlistIDs.append(p.get_attribute('href').split('/')[-1])

        
    driver.close()
    return [userID, username, len(isVertified.strip()) > 0, userurl, followers, following, tracks, ";".join(playlistIDs)]

In [80]:
#this cell takes a long time to run (more than 114 minutes to go through 1278 users)
data = asynchronousGetWebData(userUrlList, get_user_info)

users = pd.DataFrame(data, columns=['userID', 'username', 'Vertified', 'userUrl', 'followers', 'following', 'tracks', 'playlistIDs'])
users.to_csv('users.csv', index=False)

### Collect Playlists

In [81]:
users = pd.read_csv('users.csv', index_col="userID")
print(users.shape)
users.head()

(12, 7)


,username,Vertified,userUrl,followers,following,tracks,playlistIDs
userID,,,,,,,
user-307209404,NSW17,False,https://soundcloud.com/user-307209404,34,12,44,beats;memes;thunderbird;mood-prod-chuki-beats;...
mihai-dumitru-georgescu,Mihai Dumitru Georgescu,False,https://soundcloud.com/mihai-dumitru-georgescu,807,1,"1,020",playlist1;piano-covers-popular-songs;cafe-del-...
kishimura,Kishimura,False,https://soundcloud.com/kishimura,"2,931",107,50,nightcore-delete-from-yt;gucci;rumors;game-over
djmonu-2,Djmonu,False,https://soundcloud.com/djmonu-2,160,21,19,NaN
kristel-star,Merilin,False,https://soundcloud.com/kristel-star,19,10,33,m-y-c-o-v-e-r-s;xanny-billie-eilish-cover-1;pl...


In [83]:
users["playlistIDs"] = users["playlistIDs"].str.split(';')

In [129]:
userPlaylist = users.explode("playlistIDs").dropna(subset=["playlistIDs"])

In [132]:
listPlaylistURL = userPlaylist[['playlistIDs', 'userUrl']]
listPlaylistURL['playlistUrl'] = listPlaylistURL['userUrl'] + '/sets/' + listPlaylistURL['playlistIDs']
listPlaylistURL.reset_index(inplace=True)

/tmp/ipykernel_2778/5826554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listPlaylistURL['playlistUrl'] = listPlaylistURL['userUrl'] + '/sets/' + listPlaylistURL['playlistIDs']


In [136]:
def getPlayListInfo(urlPlaylistInfo):
    userID ,urlPlaylist = urlPlaylistInfo[0], urlPlaylistInfo[1]
    
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(urlPlaylist)
    
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()
    
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    #check Likes 
    time.sleep(SLEEP_TIME)
    try:
        Likes = driver.find_element(By.CLASS_NAME, 'sc-ministats-likes').find_elements(By.TAG_NAME, 'span')[1].text
    except:
        Likes = None
    #check Reposts
    try:
        Reposts = driver.find_element(By.CLASS_NAME, 'sc-ministats-reposts').find_elements(By.TAG_NAME, 'span')[1].text
    except:
        Reposts = None
    #check Datetime
    try:
        Datetime = driver.find_element(By.CLASS_NAME, 'relativeTime').get_attribute('datetime')
    except:
        Datetime = None
    #check TagContent
    try:
        TagContent = driver.find_element(By.CLASS_NAME, 'sc-tagContent').text.split(',')
    except:
        TagContent = None
    

    driver.close()
    return [userID, urlPlaylist, Likes, Reposts, TagContent, Datetime]

In [137]:
data = asynchronousGetWebData(listPlaylistURL[["userID", "playlistUrl"]].values, getPlayListInfo)

playlists = pd.DataFrame(data, columns=['UserIDPlaylist' ,'PlaylistURL', 'Likes', 'Reposts', 'TagContent' , 'Uploadtime'])
playlists.to_csv('playlists.csv', index=False)

### Collect Tracks

## Exploratory Data Analysis

## References